# SPAKE2+ (P256-SHA256-HKDF-HMAC)

SageMath implementation of SPAKE2+ as described by draft-irtf-cfrg-spake2-09.

In [22]:
# SPAKE2+ (P256-SHA256-HKDF-HMAC)
# Python/SageMath implementation to generate SPAKE2+ test vectors.

import hmac

from struct import *
from hashlib import sha256

# Optional, associated data. Used when confirming keys.
AAD = ''

# w = HMF(pw)
w = 0x7741cf8c80b9bee583abac3d38daa6b807fed38b06580cb75ee85319d25fede6 

# w0s || w1s = MHF(len(pw) || pw || len(A) || A || len(B) || B)
# w0 = w0s (mod p) and w1 = w1s (mod p)
w0 = 0x4f9e28322a64f9dc7a01b282cc51e2abc4f9ed568805ca84f4ed3ef806516cf8
w1 = 0x8d73e4ca273859c873d809431d15f30e2b722007964e32699160b54fda3ee855

# P-256 constants and helper functions
px = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
py = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5

p256 = 2^256 - 2^224 + 2^192 + 2^96 - 1
b256 = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b

FF = GF(p256)
EC = EllipticCurve([FF(p256 - 3), FF(b256)])
P = EC(FF(px), FF(py))

# seed: 1.2.840.10045.3.1.7 point generation seed (M)
mx = 0x886e2f97ace46e55ba9dd7242579f2993b64e16ef3dcab95afd497333d8fa12f
my = 0x5ff355163e43ce224e0b0e65ff02ac8e5c7be09419c785e0ca547d55a12e2d20

# seed: 1.2.840.10045.3.1.7 point generation seed (N)
nx = 0xd8bbd6c639c62937b04d997f38c3770719c629d7014d49a24b4f98baa1292b49
ny = 0x07d60aa6bfade45008a636337f5168c64d9bd36034808cd564490b1e656edbe7

M = EC(FF(mx), FF(my))
N = EC(FF(nx), FF(ny))

def wrap_print(arg, *args):
    line_length = 68
    string = arg + " " + " ".join(args)
    for hunk in (string[0+i:line_length+i] for i in range(0, len(string), line_length)):
        if hunk and len(hunk.strip()) > 0:
            print hunk

def int2bin(num):
    return format(num, 'x').zfill(64).decode('hex')

def encode_point(point):
    return '\x04' + int2bin(int(point[0])) + int2bin(int(point[1]))

def print_integer(name, x):
    wrap_print(name + ' = 0x' + int2bin(int(x)).encode('hex'))

def print_point(name, point):
    wrap_print(name + ' = 0x' + encode_point(point).encode('hex'))

def pack_point(point):
    return pack_len('\x04' + int2bin(int(point[0])) + int2bin(int(point[1])))

def pack_len(bytes):
    return pack('<Q', len(bytes)) + bytes

def pack_identity(ident):
    return pack_len(ident) if ident and len(ident) > 0 else ''

def hkdf(ikm, info):
    salt = '\x00' * sha256().digest_size
    prk = hmac.new(salt, ikm, sha256).digest()
    return hmac.new(prk, info, sha256).digest()

def derive_keys(TT):
    # Ka || Ke = Hash(TT)
    sk = sha256(TT).digest()
    Ka = sk[:16]
    Ke = sk[16:]
    wrap_print('Ka = 0x' + Ka.encode('hex'))
    wrap_print('Ke = 0x' + Ke.encode('hex'))

    # KDF(nil, Ka, "ConfirmationKeys" || AAD) = KcA || KcB
    ck = hkdf(Ka, 'ConfirmationKeys' + AAD.decode('hex') + '\x01')
    KcA = ck[:16]
    KcB = ck[16:]
    wrap_print('KcA = 0x' + KcA.encode('hex'))
    wrap_print('KcB = 0x' + KcB.encode('hex'))
    
    return Ke, KcA, KcB

def spake2(A, B):
    # Print constants w0 and w1
    print_integer('w', w)

    # A picks x randomly and uniformly, and calculates X=x*P and T=w*M+X
    x = int(FF.random_element())
    X = x * P
    print_point('X', X)
    
    T = int(w) * M + X
    print_point('T', T)
    
    # B selects y randomly and uniformly, and computes Y=y*P, S=w*N+Y
    y = int(FF.random_element())
    Y = y * P
    print_point('Y', Y)
    
    S = w * N + Y
    print_point('S', S)
    
    # Both A and B calculate a group element K
    Ka = x * (S - w * N)
    Kb = y * (T - w * M)
    assert(Ka == Kb)
    K = Ka
    
    # TT = len(A) || A || len(B) || B || len(S) || S || len(T) || T || len(K) || K || len(w) || w
    TT = pack_identity(A)
    TT += pack_identity(B)
    TT += pack_point(S)
    TT += pack_point(T)
    TT += pack_point(K)
    TT += pack_len(format(int(w), 'x').decode('hex'))
    wrap_print('TT = 0x' + TT.encode('hex'))
    
    # Derive key schedule
    Ke, KcA, KcB = derive_keys(TT)

    # MAC = HMAC(KcA/KcB, TT)
    wrap_print('MAC(A) = 0x' + hmac.new(KcA, TT, sha256).hexdigest())
    wrap_print('MAC(B) = 0x' + hmac.new(KcB, TT, sha256).hexdigest())

def spake2plus(A, B):
    # Print constants w0 and w1
    print_integer('w0', w0)
    print_integer('w1', w1)
    
    # B generates L
    L = w1 * P
    print_point('L', L)

    # A generates key share X
    x = int(FF.random_element())
    X = int(x) * P + w0 * M
    print_point('X', X)

    # B generates key share Y
    y = int(FF.random_element())
    Y = int(y) * P + w0 * N
    print_point('Y', Y)

    # A computes shared keys Z, V
    Z = x * (Y - w0 * N)
    V = w1 * (Y - w0 * N)
    print_point('Z', Z)
    print_point('V', V)

    # B computes shared keys Z, V
    assert Z == y * (X - w0 * M)
    assert V == y * L

    # TT = len(A) || A || len(B) || B || len(X) || X || len(Y) || Y || len(Z) || Z || len(V) || V || len(w0) || w0
    TT = pack_identity(A)
    TT += pack_identity(B)
    TT += pack_point(X)
    TT += pack_point(Y)
    TT += pack_point(Z)
    TT += pack_point(V)
    TT += pack_len(format(int(w0), 'x').decode('hex'))
    wrap_print('TT = 0x' + TT.encode('hex'))
    
    # Derive key schedule
    Ke, KcA, KcB = derive_keys(TT)

    # MAC = HMAC(KcA/KcB, TT)
    wrap_print('MAC(A) = 0x' + hmac.new(KcA, TT, sha256).hexdigest())
    wrap_print('MAC(B) = 0x' + hmac.new(KcB, TT, sha256).hexdigest())
    
# Set A and B to None if identities are implicit.
A_identities = ['client', '']
B_identities = ['server', '']

for A in A_identities:
    for B in B_identities:
        wrap_print('\nSPAKE2(A=\'%s\', B=\'%s\'), (P256-SHA256-HKDF-HMAC)' % (A, B))
        spake2(A, B)

for A in A_identities:
    for B in B_identities:
        wrap_print('\nSPAKE2+(A=\'%s\', B=\'%s\'), (P256-SHA256-HKDF-HMAC)' % (A, B))
        spake2plus(A, B)


SPAKE2(A='client', B='server'), (P256-SHA256-HKDF-HMAC) 
w = 0x7741cf8c80b9bee583abac3d38daa6b807fed38b06580cb75ee85319d25fed
e6 
X = 0x04ac6827b3a9110d1e663bcd4f5de668da34a9f45e464e99067bbea53f1ed4
d8abbdd234c05b3a3a8a778ee47f244cca1a79acb7052d5e58530311a9af077ba179
T = 0x04e02acfbbfb081fc38b5bab999b5e25a5ffd0b1ac48eae24fcc8e49ac5e0d
8a790914419a100e205605f9862daa848e99cea455263f0c6e06bc5a911f3e10a16b
Y = 0x0413c45ab093a75c4b2a6e71f957eec3859807858325258b0fa43df5a6efd2
63c59b9c1fbfd55bc5e75fd3e7ba8af6799a99b225fe6c30e6c2a2e0ab4962136ba8
S = 0x047aad50ba7bd6a5eacbead7689f7146f1a4219fa071cce1755f80280cc6c3
a5a73cf469f2a294a0b74a5c07054585ccd447f3f633d8631f3bf43442449e9efeba
TT = 0x0600000000000000636c69656e74060000000000000073657276657241000
00000000000047aad50ba7bd6a5eacbead7689f7146f1a4219fa071cce1755f80280
cc6c3a5a73cf469f2a294a0b74a5c07054585ccd447f3f633d8631f3bf43442449e9
efeba410000000000000004e02acfbbfb081fc38b5bab999b5e25a5ffd0b1ac48eae
24fcc8e49ac5e0d8a790914419a100e205605f986

TT = 0x0600000000000000636c69656e7441000000000000000426fbedb3b9ccea9
3d609838dcc1d4baebdbb9c287763ed4cdb2d3cc76f788d3388db3da1f63e945f3f1
ba17f7b986ab9ed3170359ee406cbb40f3e3719453b15410000000000000004d4960
922990acb87809e734fed2c2ccb72fd26ed173e8207cdc6220073ac5017660788e96
db275f6edf2ba400d4e090273c24dc907d80ff9cad7f42fd9f79c3f4100000000000
0000421996ff4d9c05b2389ae05118c519679df5d6de258b31f2a17da7604c8e3c17
bb3c4aae2ae4217951aa82144cb8b677be8061f28893f70216c1e11ba2bacd50d410
000000000000004729f7c6c5bd68310345b1a10b84ea7db64c70441da2255992208b
7a8e0b39d4f0e634acf7d440b4552a41df291ac6a409f8cf5a47cec9fed5f85fea12
41379a420000000000000004f9e28322a64f9dc7a01b282cc51e2abc4f9ed568805c
a84f4ed3ef806516cf8 
Ka = 0xfd19104b836b0ba9dfaaeab88610be57 
Ke = 0x90337374f974f673707de5ba1b98e5b8 
KcA = 0x2e10249c566677c8826b48ad10b19bb5 
KcB = 0x4fcaf8fd0bfcaeeabb9d6f48e264e4a3 
MAC(A) = 0xaaef200ea5f5c41e1fdb9b3455dde715cd8aa96f8afd3274f7159c3c5
4887f2c 
MAC(B) = 0x926eadbf4b720b46ea622d7100e0013eb2